# 4.8 Task

## 1. Create a new notebook for this task. Be sure to import the relevant libraries, along with your ords_prods_merge dataframe, which should include your newly derived columns from the previous Exercise.

In [1]:
# Import libraries

import pandas as pd
import numpy as np
import os

In [2]:
#set path

path = r'C:\Users\Karen\Documents\Personal\Data Analytics\Career Foundry\Achievement 4\Instacart Basket Analysis'

In [ ]:
# Import data frame pickle

ords_prods_merge = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_merged_updated.pkl'))

In [ ]:
# Data frame checks

ords_prods_merge.head()

In [ ]:
ords_prods_merge.shape

## 2. In this Exercise, you learned how to find the aggregated mean of the “order_number” column grouped by “department_id” for a subset of your dataframe. Now, repeat this process for the entire dataframe.

In [ ]:
ords_prods_merge.groupby('department_id').agg({'order_number': ['mean']})

## 3. Analyze the result. How do the results for the entire dataframe differ from those of the subset? Include your comments in a markdown cell below the executed code.

### The subset that only contained the first 1,000,000 rows of the data frame only returned means for 8 departments, where as looking at the mean for the entire data frame returns means for 21 departments. Also, the departments that show up in both the subset and the entire data frame, have different mean results because in the subset the mean calculation can only be done using the first 1,000,000 rows, so lots of data is left out of the calculation. However, calculating the means for the entire data frame uses every row for each department to find the means, giving a more accurate number.

## 4. Follow the instructions in the Exercise for creating a loyalty flag for existing customers using the transform() and loc() functions.

In [ ]:
ords_prods_merge['max_order'] = ords_prods_merge.groupby(['user_id'])['order_number'].transform(np.max)

In [ ]:
# Checking output of above function.

ords_prods_merge[['order_id', 'user_id', 'order_number', 'max_order']].head(100)

In [ ]:
# Taking your advice to look into why max orders doesn't seem to be matching the highest order number for each customer. 
# First, I'm remvoing the limit on number of rows shown.

pd.options.display.max_rows = None

In [ ]:
# Looking at 200 records sorted by user_id and order_number in descending order.

ords_prods_merge[['user_id', 'order_number', 'max_order']].sort_values(by=['user_id', 'order_number'], ascending=[False, False]).head(200)

### As far as I can see from the records shown above, the max_order column seems to match the highest order number. It would take many more than 200 records to really see the pattern because each item in each order is what constitutes a record and so order numbers are repeated for each item.

In [ ]:
# Creating a loyalty flag column

ords_prods_merge.loc[ords_prods_merge['max_order'] > 40, 'loyalty_flag'] = 'Loyal customer'
ords_prods_merge.loc[(ords_prods_merge['max_order'] <= 40) & (ords_prods_merge['max_order'] > 10), 'loyalty_flag'] = 'Regular customer'
ords_prods_merge.loc[ords_prods_merge['max_order'] <= 10, 'loyalty_flag'] = 'New customer'

In [ ]:
# Checking counts for new column

ords_prods_merge['loyalty_flag'].value_counts(dropna = False)

## 5. The marketing team at Instacart wants to know whether there’s a difference between the spending habits of the three types of customers you identified. Use the loyalty flag you created and check the basic statistics of the product prices for each loyalty category (Loyal Customer, Regular Customer, and New Customer). What you’re trying to determine is whether the prices of products purchased by loyal customers differ from those purchased by regular or new customers.

In [ ]:
# Grouping by 'loyalty_flag' and then aggregating mean, min, and max

ords_prods_merge.groupby('loyalty_flag').agg({'prices': ['mean', 'min', 'max']})

### The "loyal customer" category has the lowest priced mean orders. The "new customer" category has the highest priced mean orders. And the "regular customer" category has a mean order price in between the other two categories. This could be because the loyal customers have the most orders and so their order totals have to be divided by are larger number of orders. Where as new customers have the fewest orders and so the mean of their order totals would be divided by a smaller number of orders. (And the regular customers would be in between.) Over time, with large and small orders adding up, this could account for the difference in the means between these 3 groups. Doing a t-test may help determine if these differences are statistically significant.

## 6. The team now wants to target different types of spenders in their marketing campaigns. This can be achieved by looking at the prices of the items people are buying. Create a spending flag for each user based on the average price across all their orders using the following criteria:

### If the mean of the prices of products purchased by a user is lower than 10, then flag them as a “Low spender.”

### If the mean of the prices of products purchased by a user is higher than or equal to 10, then flag them as a “High spender.”

In [ ]:
# Creating new column 'mean_order_price,' then grouping by 'user_id' and finding the mean of 'prices'

ords_prods_merge['mean_order_price'] = ords_prods_merge.groupby(['user_id'])['prices'].transform(np.mean)

In [ ]:
# Checking output of above function

ords_prods_merge[['user_id', 'mean_order_price']].head(20)

In [ ]:
# Creating a 'spending_flag' column with flags for each customer

ords_prods_merge.loc[ords_prods_merge['mean_order_price'] < 10, 'spending_flag'] = 'Low spender'
ords_prods_merge.loc[ords_prods_merge['mean_order_price'] >= 10, 'spending_flag'] = 'High spender'

In [ ]:
# Checking output of above function

ords_prods_merge[['user_id', 'mean_order_price', 'spending_flag']].head(100)

In [ ]:
# Checking counts for 'spending_flag' column

ords_prods_merge['spending_flag'].value_counts(dropna = False)

## 7. In order to send relevant notifications to users within the app (for instance, asking users if they want to buy the same item again), the Instacart team wants you to determine frequent versus non-frequent customers. Create an order frequency flag that marks the regularity of a user’s ordering behavior according to the median in the “days_since_prior_order” column. The criteria for the flag should be as follows:

### If the median of “days_since_prior_order” is higher than 20, then the customer should be labeled a “Non-frequent customer.”

### If the median is higher than 10 and lower than or equal to 20, then the customer should be labeled a “Regular customer.”
### If the median is lower than or equal to 10, then the customer should be labeled a “Frequent customer.”

In [ ]:
# Creating new column 'median_days_since_prior_order,' then grouping by 'user_id' and finding the median of 'days_since_prior_order'

ords_prods_merge['median_days_since_prior_order'] = ords_prods_merge.groupby(['user_id'])['days_since_prior_order'].transform(np.median)

In [ ]:
# Checking the output of the above function

ords_prods_merge[['user_id', 'days_since_prior_order', 'median_days_since_prior_order']].head(20)

In [ ]:
# Creating a 'regularity_flag' column with flags for each customer

ords_prods_merge.loc[ords_prods_merge['median_days_since_prior_order'] > 20, 'regularity_flag'] = 'Non-frequent customer'
ords_prods_merge.loc[(ords_prods_merge['median_days_since_prior_order'] > 10) & (ords_prods_merge['median_days_since_prior_order'] <=20), 'regularity_flag'] = 'Regular customer'
ords_prods_merge.loc[ords_prods_merge['median_days_since_prior_order'] <= 10, 'regularity_flag'] = 'Frequent customer'

In [ ]:
# Checking output of above funtion

ords_prods_merge[['user_id', 'median_days_since_prior_order', 'regularity_flag']].head(30)

In [ ]:
# Checking counts for 'regularity_flag' column

ords_prods_merge['regularity_flag'].value_counts(dropna = False)

## Some errors were found in tutorial 4.9 in the prices column. It was suggested we copy and paste the fix for this into the 4.8 notebook to keep the data cleaning out of the data visualization notebook:

In [ ]:
# Looking for products that cost > $100

ords_prods_merge.loc[ords_prods_merge['prices'] > 100]

In [ ]:
# Replacing prices > $100 with NaN

ords_prods_merge.loc[ords_prods_merge['prices'] >100, 'prices'] = np.nan

In [ ]:
# Checking max price

ords_prods_merge['prices'].max()

## 8. Ensure your notebook is clean and structured and that your code is well commented.
## 9. Export your dataframe as a pickle file and store it correctly in your “Prepared Data” folder.

In [ ]:
ords_prods_merge.to_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'ords_prods_merge_flagged.pkl'))